# AI Project 3 - N Puzzle
Baosheng Feng, G22770384

In [1]:
import numpy as np
import copy
import sys
import heapq
import time

## 1. Relative Functions 

In [2]:
#No.1 heuristic function
#here, number of misplaced tiles is the used as the heuristic function
#the time is larger than manhattan distance
#this heuristic function is not used in this project

def heuristic_function_1(puzzle):
    
    sum = 0
    
    n = len(puzzle[0])
    num = 1

    for i in range(n):
        for j in range(n):
            
            if i == n-1 and j == n-1:
                if puzzle[i][j] != 0:
                    sum +=1
            else:
                if puzzle[i][j] != num:
                    sum +=1
                num +=1
    
    return sum

In [3]:
#No.2 heuristic function
#here, the 2d manhattan distance is used as the heuristic function
#this heuristic function is adjusted to no.3 heuristic function

def heuristic_function_2(puzzle):
    
    sum = 0
    n = len(puzzle[0])
    
    for i in range(n):
        for j in range(n):
                        
            if puzzle[i][j] == 0:
                continue
            else:
                if puzzle[i][j] % n == 0:
                    row = int(puzzle[i][j]/n) - 1
                else:
                    row = int(puzzle[i][j]/n)
                col = puzzle[i][j] - row*n - 1
                sum += abs(i-row) + abs(j-col)
    
    return sum

In [4]:
#No.3 heuristic function
#here, the 2d manhattan distance + linear conflict
#here, we use this heuristic function in the project

def heuristic_function(puzzle):
    
    sum = 0
    n = len(puzzle[0])
    
    for i in range(n):
        list = []        
        for j in range(n):
            
            if puzzle[i][j] == 0:
                continue
            else:
                #row
                if puzzle[i][j] % n == 0:
                    row = int(puzzle[i][j]/n) - 1
                else:
                    row = int(puzzle[i][j]/n)
                #col   
                col = puzzle[i][j] - row*n - 1
                
                #manhattan distance
                sum += abs(i-row) + abs(j-col)
                
                #linear conflict
                #here we combined linear conflict with manhattan distance
                if row == i and col != j:
                    
                    list.append([j,puzzle[i][j]])
                    
                    for item in list:
                        if item[0]<j and item[1]>puzzle[i][j]:
                            sum +=2
                            list.remove(item)
                            list.remove([j,puzzle[i][j]])
                            break                    
    return sum

In [5]:
#class of each puzzle game
#used to save all information needed in project

class Puzzle:
    def __init__(self, puzzle):
        
        self.puzzle = puzzle
        self.n = len(puzzle[0])
        self.index = None
        self.hash = hash(str(puzzle))
        
        self.g = 0
        self.h = heuristic_function(puzzle)
        
        self.zero_row = None
        self.zero_col = None
        
        self.parent = None
        self.up = None
        self.down = None
        self.left = None
        self.right = None
        self.child = []

In [6]:
# check is the puzzle is solved
def check_puzzle(puzzle):
    return np.array_equal(puzzle.puzzle, final_state)

In [7]:
#null(or the zero) tile move function
def move(puzzle):
    
    n = puzzle.n
    zero_row = puzzle.zero_row
    zero_col = puzzle.zero_col
        
    # zero move up
    # here, we create a new game board and change the position of zero and garget tile
    if zero_row != 0:
        move_up = np.copy(puzzle.puzzle)
        move_up[zero_row][zero_col] = move_up[zero_row-1][zero_col]
        move_up[zero_row-1][zero_col] = 0
        
        up = Puzzle(move_up)
        up.g = puzzle.g+1
        all_puzzle.append(up)
        up.index = len(all_puzzle)-1
        up.parent = puzzle.index
        up.zero_row = zero_row-1
        up.zero_col = zero_col
        
        puzzle.up = up.index
        puzzle.child.append(up.index)
    
    # move down
    if zero_row != n-1:
        move_down = np.copy(puzzle.puzzle)
        move_down[zero_row][zero_col] = move_down[zero_row+1][zero_col]
        move_down[zero_row+1][zero_col] = 0
        
        down = Puzzle(move_down)
        down.g = puzzle.g+1
        all_puzzle.append(down)
        down.index = len(all_puzzle)-1
        down.parent = puzzle.index
        down.zero_row = zero_row+1
        down.zero_col = zero_col
        
        puzzle.down = down.index
        puzzle.child.append(down.index)
        
    # movw left
    if zero_col != 0:
        move_left = np.copy(puzzle.puzzle)
        move_left[zero_row][zero_col] = move_left[zero_row][zero_col-1]
        move_left[zero_row][zero_col-1] = 0
        
        left = Puzzle(move_left)
        left.g = puzzle.g+1
        all_puzzle.append(left)
        left.index = len(all_puzzle)-1
        left.parent = puzzle.index
        left.zero_row = zero_row
        left.zero_col = zero_col-1
        
        puzzle.left = left.index
        puzzle.child.append(left.index)
        
    #move right
    if zero_col != n-1:
        move_right = np.copy(puzzle.puzzle)
        move_right[zero_row][zero_col] = move_right[zero_row][zero_col+1]
        move_right[zero_row][zero_col+1] = 0
        
        right = Puzzle(move_right)
        right.g = puzzle.g+1
        all_puzzle.append(right)
        right.index = len(all_puzzle)-1
        right.parent = puzzle.index
        right.zero_row = zero_row
        right.zero_col = zero_col+1
        
        puzzle.right = right.index
        puzzle.child.append(right.index)

In [8]:
#2d mahattan+linear conflict A* algorithm
def puzzle_solver_astar(puzzle):
    
    open_list = []
    close_list = set()
    result = []
    
    current = puzzle
    heapq.heappush(open_list,(current.g+current.h, current.index))
    step = 0
    
    while(open_list):
                
        current = all_puzzle[heapq.heappop(open_list)[1]]
        close_list.add(current.hash)
                        
        if check_puzzle(current):
            while current.parent:
                result.append(current.puzzle)
                current = all_puzzle[current.parent]
            result.append(current.puzzle)
            result.append(all_puzzle[current.parent].puzzle)
            result.reverse()
            return result
        
        move(current)
        child_list = current.child
                
        for num in child_list:
            
            child = all_puzzle[num]
            
            #here,hash function is used to find if the board has already been visited
            if child.hash in close_list:
                continue

            heapq.heappush(open_list,(child.g+child.h, child.index))
                  
    return result

In [9]:
#IDA* algorithm for 15 puzzle
#https://www.youtube.com/watch?v=5LMXQ1NGHwU
#https://algorithmsinsight.wordpress.com/graph-theory-2/ida-star-algorithm-in-general/
#https://en.wikipedia.org/wiki/Iterative_deepening_A*

#with the above websites, we write the following algorithm
#path_hash list is used to find is the board has already been visited

def search(path_index, path_hash, g, bound):
    
    node = all_puzzle[path_index[-1]]
    f = g + node.h
    
    if f > bound:
        return f
    if check_puzzle(node):
        return "FOUND"
    
    min = sys.maxsize
    move(node)
    
    for item in node.child:
        
        child = all_puzzle[item]
        
        if child.hash not in path_hash:
            
            path_index.append(child.index)
            path_hash.add(child.hash)
            
            t = search(path_index,path_hash,g+1,bound)
            if t == "FOUND":
                return "FOUND"
            if t < min:
                min = t
            
            path_index.remove(path_index[-1])
            path_hash.remove(child.hash)
    
    return min
        

def puzzle_solver_idastar(puzzle):
    bound = puzzle.h
    path_index = []
    path_hash = set()
    path_index.append(puzzle.index)
    path_hash.add(puzzle.hash)
    
    while(1):
        print("Current bound: " + str(bound))
        
        t = search(path_index,path_hash,0,bound)
        if t == "FOUND":
            return path_index
        if t == sys.maxsize:
            return "Not Found"
        bound = t

## 2. Main Program

In [10]:
#read from text file
#change the path, different file can be imported
path = "8-puzzle.txt"
puzzle = np.loadtxt(path, dtype=int)
print(puzzle)

[[0 3 8]
 [4 1 7]
 [2 6 5]]


In [11]:
#create the class
puzzle = Puzzle(puzzle)
print(puzzle.h)

16


In [12]:
#find zero place
for i in range(puzzle.n):
    for j in range(puzzle.n):
        if puzzle.puzzle[i][j] == 0:
            puzzle.zero_row=i
            puzzle.zero_col=j

print(puzzle.zero_row)
print(puzzle.zero_col)

0
0


In [13]:
#create a list to store all puzzle state
all_puzzle = []
all_puzzle.append(puzzle)
puzzle.index = len(all_puzzle)-1
print(puzzle.index)
print(all_puzzle[puzzle.index].puzzle)

0
[[0 3 8]
 [4 1 7]
 [2 6 5]]


In [14]:
#the final state
final_state = np.zeros((puzzle.n,puzzle.n),dtype=int)
n = 1
for i in range(puzzle.n):
    for j in range(puzzle.n):
        final_state[i][j] = n
        n += 1
final_state[puzzle.n-1][puzzle.n-1]=0
print(final_state)      

[[1 2 3]
 [4 5 6]
 [7 8 0]]


## Result

In [15]:
#IDA* algorithm

start_time = time.time()
path = puzzle_solver_idastar(puzzle)
end_time = time.time()

print("Time: " + str(end_time-start_time))

step = 0

for item in path:
    print("Step: "+str(step))
    print(all_puzzle[item].puzzle)
    step+=1


Current bound: 16
Current bound: 18
Current bound: 20
Current bound: 22
Current bound: 24
Time: 13.515413045883179
Step: 0
[[0 3 8]
 [4 1 7]
 [2 6 5]]
Step: 1
[[4 3 8]
 [0 1 7]
 [2 6 5]]
Step: 2
[[4 3 8]
 [2 1 7]
 [0 6 5]]
Step: 3
[[4 3 8]
 [2 1 7]
 [6 0 5]]
Step: 4
[[4 3 8]
 [2 0 7]
 [6 1 5]]
Step: 5
[[4 3 8]
 [2 7 0]
 [6 1 5]]
Step: 6
[[4 3 0]
 [2 7 8]
 [6 1 5]]
Step: 7
[[4 0 3]
 [2 7 8]
 [6 1 5]]
Step: 8
[[0 4 3]
 [2 7 8]
 [6 1 5]]
Step: 9
[[2 4 3]
 [0 7 8]
 [6 1 5]]
Step: 10
[[2 4 3]
 [6 7 8]
 [0 1 5]]
Step: 11
[[2 4 3]
 [6 7 8]
 [1 0 5]]
Step: 12
[[2 4 3]
 [6 0 8]
 [1 7 5]]
Step: 13
[[2 4 3]
 [0 6 8]
 [1 7 5]]
Step: 14
[[2 4 3]
 [1 6 8]
 [0 7 5]]
Step: 15
[[2 4 3]
 [1 6 8]
 [7 0 5]]
Step: 16
[[2 4 3]
 [1 6 8]
 [7 5 0]]
Step: 17
[[2 4 3]
 [1 6 0]
 [7 5 8]]
Step: 18
[[2 4 3]
 [1 0 6]
 [7 5 8]]
Step: 19
[[2 0 3]
 [1 4 6]
 [7 5 8]]
Step: 20
[[0 2 3]
 [1 4 6]
 [7 5 8]]
Step: 21
[[1 2 3]
 [0 4 6]
 [7 5 8]]
Step: 22
[[1 2 3]
 [4 0 6]
 [7 5 8]]
Step: 23
[[1 2 3]
 [4 5 6]
 [7 0 8]]
Step: 2

In [16]:
#A* algorithm

start_time = time.time()
result = puzzle_solver_astar(puzzle)
end_time = time.time()

print("Time: " + str(end_time-start_time))

step = 0

for item in result:
    print("Step: " + str(step))
    print(item)
    step +=1

Time: 0.9790239334106445
Step: 0
[[0 3 8]
 [4 1 7]
 [2 6 5]]
Step: 1
[[4 3 8]
 [0 1 7]
 [2 6 5]]
Step: 2
[[4 3 8]
 [2 1 7]
 [0 6 5]]
Step: 3
[[4 3 8]
 [2 1 7]
 [6 0 5]]
Step: 4
[[4 3 8]
 [2 0 7]
 [6 1 5]]
Step: 5
[[4 3 8]
 [2 7 0]
 [6 1 5]]
Step: 6
[[4 3 0]
 [2 7 8]
 [6 1 5]]
Step: 7
[[4 0 3]
 [2 7 8]
 [6 1 5]]
Step: 8
[[0 4 3]
 [2 7 8]
 [6 1 5]]
Step: 9
[[2 4 3]
 [0 7 8]
 [6 1 5]]
Step: 10
[[2 4 3]
 [6 7 8]
 [0 1 5]]
Step: 11
[[2 4 3]
 [6 7 8]
 [1 0 5]]
Step: 12
[[2 4 3]
 [6 0 8]
 [1 7 5]]
Step: 13
[[2 4 3]
 [0 6 8]
 [1 7 5]]
Step: 14
[[2 4 3]
 [1 6 8]
 [0 7 5]]
Step: 15
[[2 4 3]
 [1 6 8]
 [7 0 5]]
Step: 16
[[2 4 3]
 [1 6 8]
 [7 5 0]]
Step: 17
[[2 4 3]
 [1 6 0]
 [7 5 8]]
Step: 18
[[2 4 3]
 [1 0 6]
 [7 5 8]]
Step: 19
[[2 0 3]
 [1 4 6]
 [7 5 8]]
Step: 20
[[0 2 3]
 [1 4 6]
 [7 5 8]]
Step: 21
[[1 2 3]
 [0 4 6]
 [7 5 8]]
Step: 22
[[1 2 3]
 [4 0 6]
 [7 5 8]]
Step: 23
[[1 2 3]
 [4 5 6]
 [7 0 8]]
Step: 24
[[1 2 3]
 [4 5 6]
 [7 8 0]]
